In [116]:
import pandas as pd
import requests
from config import data as userdata
import os
import time

In [112]:
def getRealTimeData(city='bj', datatype='airquality', day='2018-04-01'):
    datadir = './realTimeData'
    if not os.path.isdir(datadir):
        os.mkdir(datadir)
    url = 'https://biendata.com/competition/' + datatype + '/' + city +  '/' + day + '-0/' + day + '-23/2k0d1d8'
    filename = datadir + "/" + city + '_' + datatype + '_' + day + '.csv'
    if os.path.exists(filename):
        return filename
    responses= requests.get(url)
    if responses.text[0] != '{':
        with open(filename,'w') as f:
            f.write(responses.text)
        return filename
    else:
        print('error:\n',responses.text)
        return ''

In [40]:
getRealTimeData(day='2018-04-13')

'./realTimeData/bj_airquality_2018-04-13.csv'

In [26]:
bjdata = pd.read_csv("bj_airquality_2018-04-01-0-2018-04-01-23.csv")
bjdata.head()

,id,station_id,time,PM25_Concentration,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration
0,2942426,dongsi_aq,2018-04-01 02:00:00,259.0,NaN,98.0,3.4,56.0,57.0
1,2942427,tiantan_aq,2018-04-01 02:00:00,250.0,NaN,95.0,3.1,64.0,28.0
2,2942428,guanyuan_aq,2018-04-01 02:00:00,240.0,246.0,107.0,2.2,49.0,30.0
3,2942429,wanshouxigong_aq,2018-04-01 02:00:00,255.0,260.0,87.0,2.5,65.0,35.0
4,2942430,aotizhongxin_aq,2018-04-01 02:00:00,266.0,NaN,111.0,3.5,45.0,57.0


# 輸出答案

要預測未來兩天 48小時 的 PM2.5 PM10 O3 

> 我想 4/11 送答案就是預測 4/12 ~ 4/13 的空氣品質

總共有 48 個點的數值需要預測

每個點要有 48 個小時的資料

站點包含 北京與倫敦 的點

id 格式 站點名#hours

In [123]:
sample = pd.read_csv('sample_submission.csv')
print(sample.shape)
sample.head()

(2304, 4)


,test_id,PM2.5,PM10,O3
0,dongsi_aq#0,0,0,0
1,dongsi_aq#1,0,0,0
2,dongsi_aq#2,0,0,0
3,dongsi_aq#3,0,0,0
4,dongsi_aq#4,0,0,0


In [149]:
sample.loc[sample['test_id'] == 'dongsi_aq#8','PM2.5'] = 0

print(sample.loc[sample['test_id'] == 'dongsi_aq#8'])

       test_id  PM2.5  PM10  O3
8  dongsi_aq#8      0     0   0


In [54]:
sample_id = sample['test_id']
stations = list()
for id in sample_id.iloc[:]:
    [station, hours] = id.split('#')
    if station not in stations:
        stations.append(station)
print(len(stations))
print(stations)

48
['dongsi_aq', 'tiantan_aq', 'guanyuan_aq', 'wanshouxig_aq', 'aotizhongx_aq', 'nongzhangu_aq', 'wanliu_aq', 'beibuxinqu_aq', 'zhiwuyuan_aq', 'fengtaihua_aq', 'yungang_aq', 'gucheng_aq', 'fangshan_aq', 'daxing_aq', 'yizhuang_aq', 'tongzhou_aq', 'shunyi_aq', 'pingchang_aq', 'mentougou_aq', 'pinggu_aq', 'huairou_aq', 'miyun_aq', 'yanqin_aq', 'dingling_aq', 'badaling_aq', 'miyunshuik_aq', 'donggaocun_aq', 'yongledian_aq', 'yufa_aq', 'liulihe_aq', 'qianmen_aq', 'yongdingme_aq', 'xizhimenbe_aq', 'nansanhuan_aq', 'dongsihuan_aq', 'CD1', 'BL0', 'GR4', 'MY7', 'HV1', 'GN3', 'GR9', 'LW2', 'GN0', 'KF1', 'CD9', 'ST5', 'TH4']


In [108]:
realdata = pd.read_csv(getRealTimeData(day='2018-04-13'))
realdata = realdata['station_id']
realdata_stations = list()
for station in realdata.iloc[:]:
    if station not in realdata_stations:
        realdata_stations.append(station)

print(len(realdata_stations))
print(realdata_stations)

35
['dongsi_aq', 'tiantan_aq', 'guanyuan_aq', 'wanshouxigong_aq', 'aotizhongxin_aq', 'nongzhanguan_aq', 'wanliu_aq', 'beibuxinqu_aq', 'zhiwuyuan_aq', 'fengtaihuayuan_aq', 'yungang_aq', 'gucheng_aq', 'fangshan_aq', 'daxing_aq', 'yizhuang_aq', 'tongzhou_aq', 'shunyi_aq', 'pingchang_aq', 'mentougou_aq', 'pinggu_aq', 'huairou_aq', 'miyun_aq', 'yanqin_aq', 'dingling_aq', 'badaling_aq', 'miyunshuiku_aq', 'donggaocun_aq', 'yongledian_aq', 'yufa_aq', 'liulihe_aq', 'qianmen_aq', 'yongdingmennei_aq', 'xizhimenbei_aq', 'nansanhuan_aq', 'dongsihuan_aq']


In [154]:
def injectAnswer(sample, fromDate, answer):
    formatDate = "%Y-%m-%d %H:%M:%S"
    fromDate = time.mktime(time.strptime(fromDate, formatDate))
    [rows, cols] = answer.shape
    for i in range(rows):
        data = answer.iloc[i]
        thisDate = time.mktime(time.strptime(data['time'],formatDate))
        thisDate = int((thisDate - fromDate) / (60*60))
        thisid = data['station_id'] + '#' + str(thisDate)
        sample.loc[sample['test_id'] == thisid, 'PM2.5'] = data['PM25_Concentration']
        sample.loc[sample['test_id'] == thisid, 'PM10'] = data['PM10_Concentration']
        sample.loc[sample['test_id'] == thisid, 'O3'] = data['O3_Concentration']

In [159]:
realdata = pd.read_csv(getRealTimeData(city='ld',day='2018-04-13'))

fromDate = '2018-04-12 00:00:00'

injectAnswer(sample, fromDate, realdata)

In [167]:
sample2 = sample.fillna(sample.mean())
sample2[2100:2151]
sample2.to_csv('./answer.csv', index=False, header=True)

In [58]:
realdata = pd.read_csv(getRealTimeData(city='ld',day='2018-04-13'))
realdata = realdata['station_id']
realdata_stations = list()
for station in realdata.iloc[:]:
    if station not in realdata_stations:
        realdata_stations.append(station)

print(len(realdata_stations))
print(realdata_stations)

19
['CD1', 'BL0', 'GR4', 'MY7', 'HV1', 'GN3', 'GR9', 'LW2', 'GN0', 'KF1', 'CD9', 'CT3', 'CT2', 'BX9', 'BX1', 'TH4', 'ST5', 'RB7', 'TD5']


# 思考策略

先考慮如何建立模型

輸入是歷史資料, 輸出則是兩天的未來預測

考慮到訓練參數出來

把一年的歷史資料 分成訓練與測試



# SMAPE

計分方式

In [59]:
def SMAPE(actual=list(), forecast=list()):
    if len(actual)!=len(forecast):
        return 2
    
    total = 0
    for i in range(len(actual)):
        tmp = (abs(forecast[i]) + abs(actual[i]))/2
        if tmp != 0:
            tmp = abs(forecast[i] - actual[i]) / tmp
        total += tmp
    
    return total/len(actual)

In [61]:
SMAPE([1,2,3],[1,1,1])

0.5555555555555555